<a href="https://colab.research.google.com/github/libra945/myassistant/blob/main/hello.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Speak to Text

Record audio from your microphone

In [5]:
!pip install ffmpeg-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [10]:
audio, sr = get_audio()

IndexError: ignored

**Try SpeechRecognition**

In [ ]:
!pip install SpeechRecognition

In [ ]:
import speech_recognition as sr
r = sr.Recognizer()  
with sr.Microphone() as source:
    audio = r.listen(source)

try:
    print("You said " + r.recognize(audio))
except LookupError:  
    print("Could not understand audio")  

# Hello API

**install flask and flask-ngrok**

In [2]:
!pip install pyngrok==4.1.1
!pip install flask-ngrok
!ngrok authtoken '2LiUt3LGEw3Zy9hMgtdaPFrXgu4_YSptGsNnAB9PNqXgxaEh'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15982 sha256=898e2a4cbf108d239c5421f6edb5bfb6bc0fb703deab39fa1b261aa0bbb7eabf
  Stored in directory: /root/.cache/pip/wheels/5e/0a/51/8cb053ccd84481dd3233eba4cdb608bc7a885fd8ca418c0806
Successfully built pyngrok
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


Hello World

In [4]:
from flask_ngrok import run_with_ngrok
from flask import request, jsonify, Flask
app = Flask(__name__)
app.config['JSON_AS_ASCII'] = False # using UTF-8
run_with_ngrok(app)

headers = {
    'ngrok-skip-browser-warning': '0'
    }

# 設定產品
items = [
    {"name": "4G暢打資費299型", "price": 299  },
    {"name": "4G暢打資費499型", "price": 499  },
    {"name": "4G暢打資費799型", "price": 799  },
    {"name": "4G手機資費198型", "price": 198  },
    {"name": "4G手機資費398型", "price": 398  },
    {"name": "4G手機資費598型", "price": 598  },
    {"name": "4G手機資費798型", "price": 798  },
    {"name": "4G上網資費296型", "price": 298  },
    {"name": "4G上網資費496型", "price": 498  }
]

@app.route('/checkservice')
def checkservice():
  return 'OK'

@app.route("/listall")
def listall():
  return jsonify(items)

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://4373-34-87-16-231.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [14/Feb/2023 07:34:59] "GET /checkservice HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Feb/2023 07:35:11] "GET /listall HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Feb/2023 07:37:06] "GET /checkservice HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Feb/2023 07:38:46] "GET /checkservice HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Feb/2023 07:39:50] "GET /checkservice HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Feb/2023 07:42:48] "GET /checkservice HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Feb/2023 07:43:56] "GET /listall HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Feb/2023 07:49:40] "GET /listall HTTP/1.1" 200 -
